In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf

from utils.data_utils import img_to_array, array_to_img
from transformer import spatial_transformer_network as stn

/Users/kevin/envs/tensorF/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
DIMS = (600, 600)
CAT1 = 'cat1.jpg'
CAT2 = 'cat2.jpg'
CAT3 = 'cat3.jpg'
CAT4 = 'cat4.jpg'

In [3]:
def load_data(dims, img_name, view=False):
    """
    Util function for processing RGB image into 4D tensor.

    Returns tensor of shape (1, H, W, C)
    """
    # PIL takes width then height
    dims = (dims[1], dims[0])
    image_path = './data/' + img_name
    x = img_to_array(image_path, dims, view=view)
    return x

## Load Images

In [11]:
# out dims
out_H = 600
out_W = 600
out_dims = (out_H, out_W)

# load 4 cat images
img1 = load_data(DIMS, CAT1, view=True)
img2 = load_data(DIMS, CAT2)
img3 = load_data(DIMS, CAT3)
img4 = load_data(DIMS, CAT4)

In [5]:
input_img = np.concatenate([img1, img2, img3, img4], axis=0)
B, H, W, C = input_img.shape
print("Input Img Shape: {}".format(input_img.shape))

Input Img Shape: (4, 600, 600, 3)


## Testing it Out

We'll be using an identity transform as a sanity check. This means the output image should look like the input image.

In [8]:
# 45 degree rotation
theta = np.array([[np.cos(45), -np.sin(45), 0], [np.sin(45), np.cos(45), 0]])

# # identity transform
# theta = np.array([[1., 0, 0], [0, 1., 0]])

In [9]:
x = tf.placeholder(tf.float32, [None, H, W, C])

# create localisation network and convolutional layer
with tf.variable_scope('spatial_transformer_0'):

    # create a fully-connected layer with 6 output nodes
    n_fc = 6
    W_fc1 = tf.Variable(tf.zeros([H*W*C, n_fc]), name='W_fc1')

    # affine transformation
    theta = theta.astype('float32')
    theta = theta.flatten()

    b_fc1 = tf.Variable(initial_value=theta, name='b_fc1')
    h_fc1 = tf.matmul(tf.zeros([B, H*W*C]), W_fc1) + b_fc1
    h_trans = stn(x, h_fc1, out_dims)

In [10]:
# run session
sess = tf.Session()
sess.run(tf.global_variables_initializer())
y = sess.run(h_trans, feed_dict={x: input_img})
print(y.shape)
y = np.reshape(y, (B, out_H, out_W, C))
y = array_to_img(y[0])
y.show()

(4, 600, 600, 3)
